### Necessary Set-up

In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import tensorflow as tf

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Embedding
from tensorflow.python.ops.losses import losses

from sklearn.cluster import KMeans
from tensorflow.python.client import device_lib



tsne_model = TSNE(learning_rate=100)

In [2]:
tf.__version__
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12551609130865645008
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3179663360
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8448255061708723872
physical_device_desc: "device: 0, name: Quadro K2200, pci bus id: 0000:02:00.0, compute capability: 5.0"
]
Num GPUs Available:  1


### The Embedder Model

#### Model Architecture, Losses and Accuracy

##### The Encoder Model

In [3]:
def build_encoder(embedding_size):
  model = Sequential()
  
  # The first encoder layer
  model.add(Dense(embedding_size*4, activation='relu'))
  
  # The second encoder layer
  model.add(Dense(embedding_size*2, activation='relu'))
  
  # The output layer
  model.add(Dense(embedding_size, activation='relu'))
  
  return model
  

##### The Decoder Model

In [4]:
def build_decoder(embedding_size, output_size):
  model = Sequential()
  
  # The first decoder layer
  model.add(Dense(embedding_size*2, activation='relu'))
  
  # The secod decoder layer
  model.add(Dense(embedding_size*4, activation='relu'))
  
  # The third decoder layer
  model.add(Dense(output_size, activation='sigmoid'))
  
  return model
  

##### Putting together the Ecnoder and Decoder into an AutoEncoder

In [5]:
def build_ae(encoder, decoder, output_size):
  
  input_tensor = Input(shape=(output_size,))
  embeddings = encoder(input_tensor)
  reconstructions = decoder(embeddings)

  auto_encoder = Model(input_tensor, reconstructions)
  
  return auto_encoder

##### The Auto-encoder Loss and Acuraccy

In [6]:
def recon_loss(x, x_hat):
  
  return tf.reduce_sum(tf.keras.losses.binary_crossentropy(x, x_hat))

In [7]:
def first_order_loss(X, Z):
    
    X = tf.cast(X, tf.float32)
    Z= tf.cast(Z, tf.float32)
    
    D = tf.linalg.diag(tf.reduce_sum(X,1))
    L = D - X ## L is laplation-matriX
    
    return 2*tf.linalg.trace(tf.matmul(tf.matmul(tf.transpose(Z),L),Z))

In [8]:
def ae_adversarial_loss(X, Z, x, x_hat, d_z0, d_z1, first_order, alpha):
  
  # Recon loss 
  reccon_loss = recon_loss(x, x_hat)
  f1_loss = first_order_loss(X, Z)
  
  if first_order =='with_f1':
      reccon_loss += alpha * f1_loss
  
  ### Loss 2 -> Same as the loss of the generator
  adversarial_loss = tf.reduce_sum(tf.keras.losses.binary_crossentropy(tf.ones_like(d_z0), d_z0)) + \
                     tf.reduce_sum(tf.keras.losses.binary_crossentropy(tf.zeros_like(d_z1), d_z1))
  
  return  reccon_loss  + 10 * adversarial_loss

In [9]:
def ae_accuracy(x, x_hat):
  round_x_hat = tf.round(x_hat)
  return tf.reduce_mean(tf.cast(tf.equal(x, round_x_hat), tf.float32))

#### Pretraining the Embedder

In [10]:
def pretrain_step_embd(X, x, encoder, decoder, auto_encoder, pre_optimizer, first_order, alpha):

  with tf.GradientTape() as pre_tape:
    z = encoder(x, training=True)
    x_hat = decoder(z, training=True)
    
    Z = encoder(X, training=True)
    
    pre_loss = recon_loss(x, x_hat) 
    
    if first_order == 'with_f1':
        pre_loss += alpha * first_order_loss(X, Z)
    
    
  pre_gradients = pre_tape.gradient(pre_loss, auto_encoder.trainable_variables)
  pre_optimizer.apply_gradients(zip(pre_gradients, auto_encoder.trainable_variables))
  
  pre_acc = ae_accuracy(x, x_hat)
  
  return tf.reduce_mean(pre_loss), pre_acc

In [11]:
def pretrain_embd(X, idxs, encoder, decoder, auto_encoder, pre_optimizer, first_order, alpha):
  np.random.shuffle(idxs)
  PRE_EPOCHS = 300
  Batch_size = 50
  
  for epoch in range(PRE_EPOCHS):
    
    epoch_losses = []
    epoch_acc = []
    
    for batch_idx in range(0, len(idxs), Batch_size):
    
      selected_idxs = idxs[batch_idx : batch_idx + Batch_size]
      adjacency_batch = X[selected_idxs, :]

      loss, accuracy= pretrain_step_embd(X, tf.cast(adjacency_batch, tf.float32), encoder, decoder, auto_encoder, pre_optimizer, first_order, alpha)
      
      epoch_losses.append(loss)
      epoch_acc.append(accuracy)
      
#     if epoch % 50 == 0:
#        print(f"Loss is {np.array(epoch_losses).mean()} and accuracy is {np.array(epoch_acc).mean()}")

### The Discriminator Model

#### Model Architecture

In [12]:
def build_discriminator(embedding_size):
  model = Sequential()
  
  # The input layer
  model.add(Input(shape=(embedding_size,)))
  
  # The first hidden layer
  model.add(Dense(25, activation='relu'))
  model.add(Dropout(0.25))
  
  # The second layer
  model.add(Dense(15, activation='relu'))
  model.add(Dropout(0.20))

  # The third layer
  model.add(Dense(6, activation='relu'))
  model.add(Dropout(0.20))
  
  model.add(Dense(1, activation = 'sigmoid'))
  
  return model

In [13]:
def disc_loss_function(d_z0, d_z1):
   
  loss_zero = tf.keras.losses.binary_crossentropy(tf.zeros_like(d_z0), d_z0) 
  loss_one = tf.keras.losses.binary_crossentropy(tf.ones_like(d_z1), d_z1)

  return tf.cast(loss_zero, tf.float32) + tf.cast(loss_one, tf.float32)

#### Joint Training in a step

In [14]:
def train_step(X, x0, x1, encoder, decoder, auto_encoder, discriminator, ae_optimizer, disc_optimizer, first_order, alpha):
  with tf.GradientTape() as ae_tape, tf.GradientTape() as disc_tape:
    
    z0 = encoder(x0, training=True)
    z1 = encoder(x1, training=True)
    
    Z = encoder(X, training=True)
    
    d_z0 = discriminator(z0, training=True)
    d_z1 = discriminator(z1, training=True)
    
    x0_hat = decoder(z0, training=True)
    x1_hat = decoder(z1, training = True)
    
       
    ae_loss = ae_adversarial_loss(X, Z, tf.concat([x0, x1], 0), tf.concat([x0_hat, x1_hat], 0), d_z0, d_z1, first_order, alpha)
    disc_loss = disc_loss_function(d_z0, d_z1)
    
    
  gradients_ae = ae_tape.gradient(ae_loss, auto_encoder.trainable_variables)
  gradients_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
  
  ae_optimizer.apply_gradients(zip(gradients_ae, auto_encoder.trainable_variables))
  disc_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))
  
  ae_acc = ae_accuracy(tf.concat([x0, x1], 0), tf.concat([x0_hat, x1_hat], 0))
  
  return tf.reduce_mean(ae_loss), ae_acc, tf.reduce_mean(disc_loss)

#### Pretrain Step for Discriminator

In [15]:
def pretrain_step_disc(x0, x1, encoder, discriminator, disc_pre_optimizer):
  
  z0 = encoder(x0)
  z1 = encoder(x1)
  
  with tf.GradientTape() as disc_tape_sep:
    
    d_z0= discriminator(z0, training=True)
    d_z1 = discriminator(z1, training=True)
       
    disc_loss = disc_loss_function(d_z0, d_z1)

    
  gradients_disc = disc_tape_sep.gradient(disc_loss, discriminator.trainable_variables)
  disc_pre_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))
  
  return tf.reduce_mean(disc_loss)

#### Pretraining the discriminator

In [16]:
def pretrain_disc(X, idxs_zeros, idxs_ones, encoder, discriminator, disc_pre_optimizer):
  
  EPOCHS = 40

  np.random.shuffle(idxs_zeros)
  np.random.shuffle(idxs_ones)
  Batch_size = 50

  for epoch in range(EPOCHS):
    for batch_idx in range(0, len(idxs_ones), Batch_size):
    
      selected_zeros = idxs_zeros[batch_idx : batch_idx + Batch_size]
      selected_ones = idxs_ones[batch_idx : batch_idx + Batch_size]
      
      x0 = X[selected_zeros]
      x1 = X[selected_ones]

      pretrain_step_disc(x0, x1, encoder, discriminator, disc_pre_optimizer)


#### The Train Loop

In [17]:
def adversarial_train(X, idxs_zeros, idxs_ones, encoder, decoder, auto_encoder, discriminator, ae_optimizer, disc_optimizer, first_order, alpha):
    EPOCHS = 500
    
    np.random.shuffle(idxs_zeros)
    np.random.shuffle(idxs_ones)
    
    Batch_size = 50

    for epoch in range(EPOCHS):
        for batch_idx in range(0, len(idxs_ones), Batch_size):
        
          selected_zeros = idxs_zeros[batch_idx : batch_idx + Batch_size]
          selected_ones = idxs_ones[batch_idx : batch_idx + Batch_size]
          
          x0 = X[selected_zeros]
          x1 = X[selected_ones]
        
          ### Joint Training
          train_step(X, tf.cast(x0, tf.float32), tf.cast(x1, tf.float32), encoder, decoder,
                                                     auto_encoder, discriminator, ae_optimizer, disc_optimizer, first_order, alpha)

### Piciking the Seeds Using Embedding

In [18]:
def get_seeds(N_CLUS, embedding, nodes, labels, nodes_zero, nodes_one, strategy, n_seeds):
    '''
    stratgey can be random, nearest, fair, re-cluster, fair_re-cluster
    '''
    
    model = KMeans(n_clusters=N_CLUS)
    model.fit(embedding)
    
    cluster_number = model.labels_  
    centers = model.cluster_centers_
    
    seed_ids = [[] for i in range(N_CLUS)]   
    
    for i in range(N_CLUS):
    
        if strategy == 'nearest':
          sorted_distance = np.array(sorted([[np.sqrt(np.sum(np.power(centers[i] - embedding[j], 2))), j] for j in range(len(embedding)) if i == cluster_number[j]]))
          seed_ids[i].extend(list(sorted_distance[:n_seeds, 1]))
        
        
        elif strategy == 're-cluster':
          temp = []
          sorted_distance = np.array(sorted([[np.sqrt(np.sum(np.power(centers[i] - embedding[j], 2))), j] for j in range(len(embedding)) if i == cluster_number[j]]))
          temp.extend(list(sorted_distance[:n_seeds, 1]))
          
          portion_zero = 0
          portion_one = 0
          
          for num in temp:
            if num in nodes_zero:
              portion_zero += 1
            elif num in nodes_one:
              portion_one += 1
          
          zero_in_clus = embedding[np.logical_and(cluster_number == i, labels == 0)]
          zero_inds = nodes[np.logical_and(cluster_number == i, labels == 0)]
          
          one_in_clus = embedding[np.logical_and(cluster_number == i, labels == 1)]
          one_inds = nodes[np.logical_and(cluster_number == i, labels == 1)]   
          
          added_to_zero = 0
          if len(zero_in_clus) != 0:
              model_on_zero = KMeans(n_clusters=1)
              model_on_zero.fit(zero_in_clus)
              center_zero = model_on_zero.cluster_centers_
              
              sorted_distance_zero = np.array(sorted([[np.sqrt(np.sum(np.power(center_zero - zero_in_clus[j], 2))), j] for j in range(len(zero_in_clus))]))
              seed_ids[i].extend([zero_inds[int(i)] for i in sorted_distance_zero[:portion_zero, 1]])
              
              added_to_zero = len(seed_ids[i])
              assert added_to_zero == portion_zero
               
          added_to_one = 0
          if len(one_in_clus) != 0:
              model_on_one = KMeans(n_clusters=1)
              model_on_one.fit(one_in_clus)
              center_one = model_on_one.cluster_centers_
              
              sorted_distance_one = np.array(sorted([[np.sqrt(np.sum(np.power(center_one - one_in_clus[j], 2))), j] for j in range(len(one_in_clus))]))
              seed_ids[i].extend([one_inds[int(i)] for i in sorted_distance_one[:portion_one, 1]])
              
              added_to_one = len(seed_ids[i]) - added_to_zero
              assert added_to_one == portion_one
              
          assert n_seeds == added_to_zero + added_to_one
          assert len(seed_ids[i]) == n_seeds
    
    return np.reshape(seed_ids, newshape=(-1, ))

### The IC algorithm

In [19]:
def IC(G, seeds, imp_prob, recover_prob = 0, remove = 0):
  
    impressed = []
    removed = []
    front = list(seeds[:])
    
    while front:
        impressed.extend(front)
        impressed = np.array(impressed)
    
        if recover_prob != 0:
    
            random_draws = np.random.uniform(size=len(impressed))
    
            if remove:
                removed.extend(impressed[random_draws < recover_prob])
                removed = list(set(removed))
    
            impressed = impressed[random_draws >= recover_prob]
              
        impressed = list(impressed)
        new_front = []
    
        for node in front:
    
            neighbours = list(G.neighbors(node))
    
            for neigh in neighbours:
    
                expr_prob = np.random.uniform(size=1)[0]
                if expr_prob < imp_prob and not (neigh in impressed) and not (neigh in new_front) and not (neigh in removed):
                    new_front.append(neigh)
    
        front = new_front[:]
     
    impressed = np.reshape(np.array(impressed), newshape=(-1,))
    
    return impressed

#### Repeated IC

In [20]:
def repeated_IC(G, nodes_zero, nodes_one, seeds, seeds_type, n_expr, imp_prob, recover_prob = 0, remove = 0):
  zeros_count = []
  ones_count = []
  total_count = []
  
  for i in range(n_expr):
    impressed = IC(G, seeds, imp_prob, recover_prob = recover_prob, remove = remove)
    total_count.append(len(impressed))
    
    count_zeros = 0
    count_ones = 0
    
    for imp in impressed:
      if imp in nodes_zero:
        count_zeros += 1
      elif imp in nodes_one:
        count_ones += 1
    
    zeros_count.append(count_zeros)
    ones_count.append(count_ones)
    
  total_imp = np.round(np.mean(total_count), 2)
  total_fraction = np.round(total_imp / len(G.nodes()), 3)
  
  fraction_zero = np.round(np.mean(zeros_count) / len(nodes_zero), 3)
  fraction_one = np.round(np.mean(ones_count) / len(nodes_one), 3)
  
  return total_imp, total_fraction, fraction_zero, fraction_one


### Loading the real graph

In [21]:
def get_graph_real():
  graph_df = pd.read_csv('edges.txt', sep="\t", header=None)
  graph_df.columns = ['s', 't']
  
  attr_df = pd.read_csv('attr.txt', sep="\t", header=None)
  attr_df.columns = ['id', 'College', 'Age', 'Major']
  
  edges = []

  for index, row in graph_df.iterrows():
    edge_cur = (row.s, row.t)

    edges.append(edge_cur)
    
  input_G = nx.from_edgelist(edges)
  
  extra_nodes = []
  for index, row in attr_df.iterrows():
    if row.Age > 20:
      extra_nodes.append(row.id)
  
  unfrozen_G = nx.Graph(input_G)
  
  for node in input_G.nodes():
    if node in extra_nodes:
      unfrozen_G.remove_node(node)
  
  X = nx.to_numpy_matrix(unfrozen_G)
  G = nx.from_numpy_matrix(X)

  return G, X, unfrozen_G

In [22]:
def get_nodes_labels_real(input_G):
  data = pd.read_csv('attr.txt', sep="\t", header=None)
  data.columns = ['id', 'College', 'Age', 'Major']

  dict_labels = {}

  for index, row in data.iterrows():
    if row.Age < 20:
      dict_labels[row.id] = 0
    elif row.Age == 20:
      dict_labels[row.id] = 1
    else:
      dict_labels[row.id] = 2

  labels = []
  for node in list(input_G.nodes()):
    labels.append(dict_labels[node])

  # Remember that whenever you want do logical operation on a sequence, that sequence should be numpy array
  labels = np.array(labels)

  nodes = np.arange(len(input_G.nodes()))

  nodes_zero = nodes[labels == 0]
  nodes_one = nodes[labels == 1]

  return nodes_zero, nodes_one, labels

In [23]:
def get_idxs(n, nodes_zero, nodes_one):
  
  diff_size = np.abs(len(nodes_one) - len(nodes_zero))
  
  idxs_zeros = nodes_zero[:]
  idxs_ones = nodes_one[:]
  rep_policy = False
  
  if len(nodes_zero) < len(nodes_one):
    if diff_size > len(nodes_zero):
      rep_policy = True
    zero_draws = np.random.choice(nodes_zero,size=diff_size, replace=rep_policy)
    idxs_zeros = np.concatenate((idxs_zeros, zero_draws))
  
  elif len(nodes_zero) > len(nodes_one):
    if diff_size > len(nodes_one):
      rep_policy = True
    one_draws = np.random.choice(nodes_one,size=diff_size, replace=rep_policy)
    idxs_ones = np.concatenate((idxs_ones, one_draws))
  
  assert len(idxs_zeros) == len(idxs_ones)
  
  return np.arange(n), idxs_zeros, idxs_ones
    

In [24]:
def print_edges(G, nodes_zero, nodes_one):
  zero_edges = 0
  one_edges = 0
  accross_edges = 0

  for (v1, v2) in G.edges():
    if v1 in nodes_zero and v2 in nodes_zero:
      zero_edges += 1
    elif v1 in nodes_one and v2 in nodes_one:
      one_edges += 1
    elif v1 in nodes_one and v2 in nodes_zero:
      accross_edges += 1
    elif v1 in nodes_zero and v2 in nodes_one:
      accross_edges += 1

  print(f" edges in zero community: {zero_edges}")
  print(f" edges in one community: {one_edges}")
  print(f" edges across communities: {accross_edges}")


### Running the experiments

In [25]:
embedding_size = 30

# Can get `with_f1` or `without_f1`
first_order_imp = 'no_f1'
alpha = 0.05

# 1. Creating the Graph and Getting the Adj Matrix
G, X, input_G = get_graph_real()
n = len(G.nodes())

# 2. Getting seperate lists for seperate communities and the label for each community
nodes_zero, nodes_one, labels = get_nodes_labels_real(input_G)

print_edges(G, nodes_zero, nodes_one)

# 3. Getting the idxs suitable for training.
idxs, idxs_zeros, idxs_ones = get_idxs(n, nodes_zero, nodes_one)

# 4. Creating the Embedder
encoder = build_encoder(embedding_size)
decoder = build_decoder(embedding_size, n)
auto_encoder = build_ae(encoder, decoder, n)

# 5. Creating the Discriminator
discriminator = build_discriminator(embedding_size)

# 6. Pretraining the Embedder and the Discriminator
pre_optimizer_embd = tf.keras.optimizers.Adam()
pre_optimizer_disc = tf.keras.optimizers.Adam()

pretrain_embd(X, idxs, encoder, decoder, auto_encoder, pre_optimizer_embd, first_order_imp, alpha)
pretrain_disc(X, idxs_zeros, idxs_ones, encoder, discriminator, pre_optimizer_disc)
# print('6')

# # 6-1. Get the pretrain-embeddings
pre_embds = encoder(X)
print('pre-training done.')

 edges in zero community: 513
 edges in one community: 7441
 edges across communities: 1706

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

pre-training done.


In [26]:
# 7. Adversarial Training
ae_optimizer = tf.keras.optimizers.Adam()
disc_optimizer = tf.keras.optimizers.Adam()

adversarial_train(X, idxs_zeros, idxs_ones, encoder, decoder, auto_encoder, discriminator, ae_optimizer, disc_optimizer, first_order_imp, alpha)

#7-1. Getting the fair-embeddings
fair_embds = encoder(X)

print('adversarial training done.')

adversarial training done.


#### Experiments settings

In [27]:
N_CLUSs = [4]

n_seedss = [1, 2, 3, 4, 5, 6, 7, 8, 10]
# n_seedss = [8]

# Methods for getting the seeds can be nearest or re-cluster
strategy = 're-cluster'

In [28]:
rows = '['
first = True


for N_CLUS in N_CLUSs:
  for n_seeds in n_seedss:
    if first:
        first = False
    else:
        rows += ',\n'
    #8. Getting the seeds for the embeddings and baselines
    fair_seeds = get_seeds(N_CLUS, fair_embds, idxs, labels, nodes_zero, nodes_one, strategy, n_seeds)
    pre_seeds = get_seeds(N_CLUS, pre_embds, idxs, labels, nodes_zero, nodes_one, strategy, n_seeds)
    
    #9. Getting the final results
    total_fair, fair_frac, zero_fair, one_fair = repeated_IC(G, nodes_zero, nodes_one, fair_seeds, 'fair', 2000, 0.01)
    total_pre, pre_frac, zero_pre, one_pre = repeated_IC(G, nodes_zero, nodes_one, pre_seeds, 'pre', 2000, 0.01)    
    
    #10. Building the current row and adding it to the rows.
    row =[ embedding_size ,  N_CLUS,  n_seeds, total_fair, fair_frac, zero_fair, one_fair,  total_pre ,  pre_frac ,  zero_pre ,  one_pre, '\'' + strategy + '\'']
    print(row)
 
    rows += '[' + ', '.join(map(str, row)) + ']'
    
rows += ']'


print(rows)


[30, 4, 1, 7.21, 0.016, 0.015, 0.017, 8.44, 0.019, 0.004, 0.023, "'re-cluster'"]
[30, 4, 2, 14.67, 0.033, 0.03, 0.034, 16.63, 0.038, 0.018, 0.043, "'re-cluster'"]
[30, 4, 3, 21.07, 0.048, 0.066, 0.043, 22.35, 0.051, 0.034, 0.055, "'re-cluster'"]
[30, 4, 4, 28.98, 0.066, 0.082, 0.061, 27.98, 0.063, 0.057, 0.065, "'re-cluster'"]
[30, 4, 5, 36.01, 0.082, 0.12, 0.071, 35.41, 0.08, 0.082, 0.08, "'re-cluster'"]
[30, 4, 6, 43.42, 0.098, 0.112, 0.095, 42.04, 0.095, 0.107, 0.092, "'re-cluster'"]
[30, 4, 7, 50.98, 0.116, 0.138, 0.109, 48.69, 0.11, 0.142, 0.102, "'re-cluster'"]
[30, 4, 8, 55.01, 0.125, 0.17, 0.112, 53.01, 0.12, 0.152, 0.111, "'re-cluster'"]
[30, 4, 10, 68.95, 0.156, 0.199, 0.144, 67.81, 0.154, 0.173, 0.148, "'re-cluster'"]
[[30, 4, 1, 7.21, 0.016, 0.015, 0.017, 8.44, 0.019, 0.004, 0.023, 're-cluster'],
[30, 4, 2, 14.67, 0.033, 0.03, 0.034, 16.63, 0.038, 0.018, 0.043, 're-cluster'],
[30, 4, 3, 21.07, 0.048, 0.066, 0.043, 22.35, 0.051, 0.034, 0.055, 're-cluster'],
[30, 4, 4, 28.98,

In [30]:
import pickle

with open('embeddings_6.pickle', 'wb') as f:
    pickle.dump([G, embedding_size, fair_embds, pre_embds, idxs, labels, nodes_zero, nodes_one], f)

print('Saved')

Saved
